In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import csv
from openpyxl import Workbook #To export data to Excel

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
#Enter basic data and URLs
page = 1

base_link = 'https://www.newsportal.hu' #This is just an example link!
login_url = 'https://www.newsportal.hu/admin/login' #This is just an example link!

In [ ]:
driver = webdriver.Chrome()
driver.get(login_url)

#Entering login data
driver.find_element(By.NAME, 'ldap_username').send_keys('own_username') #Enter your own username
driver.find_element(By.NAME, 'password').send_keys('own_password') #Enter your own password
driver.find_element(By.NAME, 'password').send_keys(Keys.RETURN)

time.sleep(10)  #Waiting for the 2FA page

#Enter the 2FA code manually
input("Type the 2FA code and press Enter....")

session = requests.Session()
cookies = driver.get_cookies()
for cookie in cookies:
    session.cookies.set(cookie['name'], cookie['value'])

driver.quit()

In [ ]:
#List of the heading names
article_types = ['Tabloid lead article', 'Afternoon lead article', 'Morning lead article', 'Early morning lead article', '-- Not lead article --']

In [ ]:
articles = []

while page != 5:
    url = f'https://www.newsportal.hu/admin/articles?page={page}'

    protexted_page = session.get(url)
    soup = bs(protexted_page.text, 'html.parser')
    tbody = soup.find('tbody')

    dates = tbody.find_all('td', class_ = 'pl-2 text-nowrap') #Publication dates of articles
    titles = tbody.find_all('div', class_ = 'text-truncate w-100 weight-700 pl-1') #Titles of articles
    robot = tbody.find_all('td', class_ = 'position-relative') #AI generated article?
    stats = tbody.find_all('td', class_ = 'text-nowrap text-center') #Numbers of clicks
    headings = tbody.find_all('a', class_ = 'label label-default weight-600 pxl-12') #Names of headings
    authors = tbody.find_all('a', class_ = 'text-dark') #Authors
    top = tbody.find_all('div', class_ = 'type') #Top (lead) article?

    lead_articles = tbody.find_all('td', title = True) #Filtering out <td> elements that have a title= attribute
    lead_types = []

    for i in range(len(lead_articles)):
        if lead_articles[i]['title'] in article_types: #If the title= attribute value is in the list, append it
            lead_types.append(lead_articles[i]['title'])

    for i in range(20):
        article = {}
        article['date'] = dates[i].get_text()
        article['title'] = titles[i].find('a')['title']
        article['url'] = base_link + titles[i].find('a')['data-url']
        article['robot'] = robot[i]['title']

        try:
            article['stat'] = int(stats[i].get_text().strip())
        except ValueError:
            article['stat'] = 0

        try:
            article['heading'] = headings[i].get_text()
        except IndexError:
            article['heading'] = 'No heading'
            
        article['author'] = authors[i].get_text()
        article['top'] = top[i]['data-type']
        article['lead'] = lead_types[i]
    
        articles.append(article)

    for article in articles:
        if article['lead'] == '-- Not lead article --':
            article['lead'] = cikk['lead'].strip('- ')

    print(f'Here we are now: {page}') #Write down the page number to check
    page += 1

In [ ]:
#Creating csv file from the data
with open ('probe.csv', 'w', newline = '', encoding = 'utf-8') as csvfile:
    writer = csv.writer(csvfile)

    for article in articles:
        writer.writerow(article.values())

In [ ]:
#Creating xlsx file from the data
workbook = Workbook()
sheet = workbook.active

sheet.append(['Date', 'Title', 'URL', 'Robot?', 'Stat', 'Heading', 'Author', 'Top', 'Lead article?'])

for article in articles:
    sheet.append(list(article.values()))

workbook.save('articles_2024.xlsx')